In [112]:
%pip -q install google-genai

In [113]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
try:
  columns = os.get_terminal_size().columns
except:
  columns = 150

In [131]:

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"
response = client.models.generate_content(
    model=MODEL_ID,
    contents='',
    config={"tools": [{"google_search": {}}]}
    # Inserir a tool de busca do Google ###############################################

)

In [114]:
# Instalar Framework ADK de agentes do Google
!pip install -q google-adk

In [115]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [116]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [117]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [118]:
# --- Agente 1: Analista Emocional --- #
def agente_analista(relato_usuario):

  analista = Agent(
    name="agente_analista",
    model="gemini-2.0-flash",
    instruction="""
    Você é um analista emocional especializado em compreender profundamente os estados afetivos humanos a partir de relatos textuais.
    Sua tarefa é interpretar o relato do usuário, identificar as emoções presentes, sua intensidade, possíveis causas e sinais de alerta.
    Primeiramente liste sentimentos como tristeza, angústia, medo, raiva, solidão, etc. Segundamente classifique como leve, moderada ou intensa.
    Terceiramente analise o contexto fornecido para entender o que pode estar provocando o estado emocional.
    Após isso, observe os sinais de alertas se a Indícios de desesperança extrema, falas sobre suicídio, automutilação,
    abandono total de atividades, etc. E por último  Elabore um pequeno parágrafo explicando o que a pessoa está sentindo,
    para orientar os próximos agentes.
    """,
    description="Agente que analisa relatos emocionais",
  )
  entrada_do_agente = f"Relato do usuário: {relato_usuario}"
  analise = call_agent(analista, entrada_do_agente)
  return analise

In [119]:
# --- Agente 2: Orientador Especialista --- #
def agente_orientador(analise_emocional):
  orientador = Agent(
    name="agente_orientador",
    model="gemini-2.0-flash",
    instruction=f"""
      Você é um especialista em estratégias de saúde mental baseadas em evidências. Sua tarefa:
      1. ANALISAR o estado emocional identificado:
          - "{analise_emocional}"
      2. GERAR um plano com:
          - 3 técnicas imediatas (ex.: respiração 4-7-8 para ansiedade)
          - 2 estratégias de médio prazo (ex.: journaling para autoconhecimento)
          - 1 ação preventiva (ex.: pausas a cada 90 minutos de trabalho)
      3. CRITÉRIOS:
          - Baseie-se em TCC, ACT ou DBT quando aplicável
          - Adapte ao contexto do usuário (ex.: se histórico mostrar resistência a meditação, sugira alternativas)
          - Evite jargões técnicos
      Formato de saída:
      **Técnicas Imediatas:**
      1. [Nome da técnica]: [Passo a passo curto] (Duração: X min)
      2. ...

      **Estratégias Contínuas:**
      - [Sugestão personalizada]
      """,
    description="Transforma análises emocionais em planos de ação práticos e baseados em evidências.",
  )
  orientacoes = call_agent(orientador, analise_emocional)
  return orientacoes

In [120]:
# --- Agente 3: Resposta Empática --- #
def agente_resposta(analise, orientacoes):
  redator = Agent(
    name="agente_resposta",
    model="gemini-2.0-flash",
    instruction=f"""
      Você é um comunicador especializado em saúde mental. Sua missão:
      1. Contextualizar:
        - Estado emocional: "{analise}"
        - Orientações técnicas: "{orientacoes}"

      2. Criar uma resposta que:
        - Valide a experiência emocional sem julgamentos
        - Explique as técnicas sugeridas com analogias cotidianas
        - Use tom de conversa (evite "você deve")
        - Inclua 1 metáfora ou exemplo concreto

      3. Formato:
        - 1 parágrafo de validação
        - 1 parágrafo com técnica principal (passo a passo simplificado)
        - Frase de encorajamento final

      Regras:
      - Nível de formalidade: amigo especialista
      - Evite clichês como "força" ou "isso vai passar"
      """,
    description="Transforma orientações técnicas em diálogos acolhedores e personalizados."
  )
  entrada = f"Análise emocional: {analise} Orientações: {orientacoes}"
  resposta = call_agent(redator, entrada)
  return resposta

In [124]:
# --- Agente 4: Validador de Resposta --- #
def agente_validador(resposta_gerada):
  validador = Agent(
    name="agente_validador",
    model="gemini-2.0-flash",
    instruction=f"""
    Você é um especialista em validação de respostas terapêuticas com foco em:

    **Critérios de Avaliação:**
    1. **Precisão Clínica**:
        - Verificar se a resposta evita diagnósticos médicos não autorizados
        - Confirmar se menciona a necessidade de avaliação profissional quando necessário
    2. **Empatia Efetiva**:
        - Analisar se o tom é acolhedor sem ser paternalista
        - Garantir que valida os sentimentos sem minimizá-los
    3. **Estrutura de Resposta**:
        - Checar se oferece: validação + informação + ação concreta
        - Verificar se evita jargões técnicos excessivos
    4. **Gatilhos Potenciais**:
        - Sinalizar qualquer linguagem que possa ser interpretada como julgamento
        - Identificar termos absolutos ("nunca", "sempre") que possam gerar culpa

    **Formato de Feedback:**
    - Para respostas adequadas: "✔ Resposta validada - Atende todos os critérios terapêuticos"
    - Para melhorias: Listar itens específicos com sugestões concretas

    **Exemplo de Boa Prática:**
    "Vejo que você está passando por um momento difícil [validação].
    A dor no peito pode ser um sintoma de ansiedade [informação],
    mas recomendo consultar um médico para descartar outras causas [ação]."
    """,
    description="""Agente que valida respostas emocionais segundo protocolos de
    comunicação terapêutica, garantindo segurança, empatia e utilidade prática."""
  )
  validacao = call_agent(validador, resposta_gerada)
  return validacao

In [132]:
# --- Agente 5: Buscador de Recursos Externos --- #
def agente_recursos(analise_emocional):
  buscador = Agent(
    name="agente_recursos",
    model="gemini-2.0-flash",
    instruction=f"""
      Busque recursos para: {analise}
    - TIPOS: vídeos, artigos, podcasts. Utileze a busca do Google.
    - FORMATO DE SAÍDA:
      obeservação: em hipótese alguma esqueça de fornecer o link
      "TÍTULO (TIPO, DURAÇÃO): DESCRIÇÃO [FONTE] → LINK"
    - EXEMPLO:
      "Respiração 4-7-8 (Vídeo, 5min): Técnica para ansiedade [Canal Bem Estar] → youtu.be/exemplo"
    - LIMITAÇÕES:
      Traga no máximo 6 resultados.
    """,
    tools=[google_search],
    description="Seleciona conteúdos confiáveis para apoio emocional"
  )
  entrada_do_agente = f"""
  Necessidade: Recursos para lidar com {analise_emocional}
  Critérios:
  - Fontes confiáveis (gov.br, who.int, canais especializados)
  - Conteúdo prático e objetivo
  - Evitar autoajuda genérica
  """
  recursos = call_agent(buscador, entrada_do_agente)
  return recursos

In [134]:
print("🧠💙 Bem-vindo ao FeelWell - Seu Sistema de Apoio Emocional 💙🧠")

relato = str(input("✍️ Por favor, compartilhe como você está se sentindo (pode ser breve ou detalhado): "))
if not relato:
  display(to_markdown("Por favor, compartilhe como você está se sentindo para podermos ajudar."))
else:
  display(to_markdown(f"Obrigado por compartilhar. Vamos analisar e te ajudar..."))
  print('-' * columns)

  analise = agente_analista(relato)
  display(to_markdown(analise))
  orientacoes = agente_orientador(analise)

  print('-' * columns)
  display(to_markdown(orientacoes))
  resposta = agente_resposta(analise, orientacoes)

  print('-' * columns)
  display(to_markdown(resposta))
  validacao = agente_validador(resposta)

  print('-' * columns)
  display(to_markdown('-- Validação da Resposta --'))
  display(to_markdown(validacao))

  print('-' * columns)
  recursos = agente_recursos(analise)
  display(to_markdown(recursos))

🧠💙 Bem-vindo ao FeelWell - Seu Sistema de Apoio Emocional 💙🧠
✍️ Por favor, compartilhe como você está se sentindo (pode ser breve ou detalhado): Tenho uma prova importante amanhã e estou sentindo muita ansiedade e estou muito nervos, por conta disso não estou conseguindo estudar adequadamente 


> Obrigado por compartilhar. Vamos analisar e te ajudar...

------------------------------------------------------------------------------------------------------------------------------------------------------


> **Sentimentos:**
> 
> *   Ansiedade: Intensa
> *   Nervosismo: Moderado
> 
> **Causas:**
> 
> *   Proximidade de uma prova importante.
> *   Dificuldade em estudar devido à ansiedade.
> 
> **Sinais de Alerta:**
> 
> *   Nenhum sinal de alerta imediato foi detectado no relato.
> 
> **Análise:**
> 
> O usuário está passando por um momento de alta ansiedade e nervosismo devido à proximidade de uma prova importante. A ansiedade está afetando sua capacidade de estudar, o que pode agravar ainda mais o quadro. É importante ajudá-lo a encontrar estratégias para lidar com a ansiedade e melhorar o foco nos estudos.


------------------------------------------------------------------------------------------------------------------------------------------------------


...

------------------------------------------------------------------------------------------------------------------------------------------------------


> -- Validação da Resposta --

> ✔ Resposta validada - Atende todos os critérios terapêuticos
> 


------------------------------------------------------------------------------------------------------------------------------------------------------


> Para te ajudar a lidar com a ansiedade e o nervosismo antes da prova, preparei alguns recursos práticos e confiáveis:
> 
> 
> Com base na sua necessidade, aqui estão alguns recursos que podem te ajudar a lidar com a ansiedade e o nervosismo antes da prova:
> 
> *   **Técnicas de Respiração:**
> 
>     *   **Respiração Diafragmática (Artigo):** A CNN Brasil explica como a respiração diafragmática (ou respiração profunda) pode ajudar a controlar a ansiedade. [CNN Brasil] → [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH8DgBVKWgetlHnUKaxgz018ByAvOcr6bUwPZS3Yj3588RJcDcVBvDgA3hl1UyNfZ86JXgvITQOU7h7vCwG7WqKYm3wlju-RlU207JeNDhrMQq8lu0DEWibcEtAajsmxwsYLuhTNLUrQozc38swCvbuzU4FPFKIvk9F-6o4T0TBN_XFotRxZ3YBli2p-icFt83DiVnGPY1REEiE8c_4m-LN4ZpPfjAm6b2Y4A==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH8DgBVKWgetlHnUKaxgz018ByAvOcr6bUwPZS3Yj3588RJcDcVBvDgA3hl1UyNfZ86JXgvITQOU7h7vCwG7WqKYm3wlju-RlU207JeNDhrMQq8lu0DEWibcEtAajsmxwsYLuhTNLUrQozc38swCvbuzU4FPFKIvk9F-6o4T0TBN_XFotRxZ3YBli2p-icFt83DiVnGPY1REEiE8c_4m-LN4ZpPfjAm6b2Y4A==)
>     *   **Respiração 4-7-8 (Artigo):** A CNN Brasil também descreve a técnica de respiração 4-7-8, um exercício de relaxamento que envolve inspirar por quatro contagens, prender a respiração por sete contagens e expirar por oito contagens. [CNN Brasil] → [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH8DgBVKWgetlHnUKaxgz018ByAvOcr6bUwPZS3Yj3588RJcDcVBvDgA3hl1UyNfZ86JXgvITQOU7h7vCwG7WqKYm3wlju-RlU207JeNDhrMQq8lu0DEWibcEtAajsmxwsYLuhTNLUrQozc38swCvbuzU4FPFKIvk9F-6o4T0TBN_XFotRxZ3YBli2p-icFt83DiVnGPY1REEiE8c_4m-LN4ZpPfjAm6b2Y4A==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXH8DgBVKWgetlHnUKaxgz018ByAvOcr6bUwPZS3Yj3588RJcDcVBvDgA3hl1UyNfZ86JXgvITQOU7h7vCwG7WqKYm3wlju-RlU207JeNDhrMQq8lu0DEWibcEtAajsmxwsYLuhTNLUrQozc38swCvbuzU4FPFKIvk9F-6o4T0TBN_XFotRxZ3YBli2p-icFt83DiVnGPY1REEiE8c_4m-LN4ZpPfjAm6b2Y4A==)
> 
> *   **Mindfulness e Meditação:**
> 
>     *   **Práticas de Mindfulness para Acadêmicos (Vídeo):** A Faculdade CENSUPEG oferece um vídeo sobre práticas de mindfulness para acadêmicos, com técnicas simples e eficazes para reduzir o estresse e aumentar a concentração. [Faculdade CENSUPEG] → [https://www.censupeg.com.br/tcc](https://www.censupeg.com.br/tcc)
>     *   **Mindfulness nos Estudos (Vídeo):** Este vídeo explora a técnica de estudos através do Mindfulness, ensinando como dominar o corpo e a mente, relaxar, manter a atenção plena e estudar no presente. [YouTube] → [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHx8BTFI4Re4AfftwcxSn_vGsshNeQOUH-GMFvax_3tKodgdhOeCfrXs_MKC4B6G2aIJVRZQVmQ-dcTBIWB9jcRPSsWEK84JDldDsEYEOuIePNIGzlSj0Gzb3kx9gs6eXMY5LW4dg==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHx8BTFI4Re4AfftwcxSn_vGsshNeQOUH-GMFvax_3tKodgdhOeCfrXs_MKC4B6G2aIJVRZQVmQ-dcTBIWB9jcRPSsWEK84JDldDsEYEOuIePNIGzlSj0Gzb3kx9gs6eXMY5LW4dg==)
> 
> *   **Podcasts:**
> 
>     *   **Podcasts para Saúde Mental de Vestibulandos (Lista):** O Educa Mais Brasil indica podcasts para ajudar na saúde mental, incluindo "É nóia minha?", que aborda ansiedade e procrastinação. [Educa Mais Brasil] → [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFxszQCLOoA0t0MhkP4g2k8_uo5Va_bri5mQ-PADmswHHjFpJDvwgNpYKY14gRA5dqEk5Wsi5CeK7b_gmuDxVuqDllWW24Lrq1mgxmRgctqlWIBmnIC9aQG9baiCM31HK5QpBe6IwOpsO2RxnPRlrhY_tzrFrlb5BY4fxMOgQx1CFFEle2dcC85eC3tC7NtD_696dcxyVXi5OmDIOONyXbsMaoLe79KOswqZIyMXoMU7w==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFxszQCLOoA0t0MhkP4g2k8_uo5Va_bri5mQ-PADmswHHjFpJDvwgNpYKY14gRA5dqEk5Wsi5CeK7b_gmuDxVuqDllWW24Lrq1mgxmRgctqlWIBmnIC9aQG9baiCM31HK5QpBe6IwOpsO2RxnPRlrhY_tzrFrlb5BY4fxMOgQx1CFFEle2dcC85eC3tC7NtD_696dcxyVXi5OmDIOONyXbsMaoLe79KOswqZIyMXoMU7w==)
>     *   **Podcasts sobre Adolescência, Escola e Saúde Mental (Lista):** A Gama Revista lista podcasts sobre saúde mental, incluindo um episódio sobre como acolher alunos com ansiedade e depressão. [Gama Revista] → [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGJsEjV6EgkuWJJq002Fo56MyrhSkDR1hfPaqBc9JnxqojIt3fKtTrPOXFZMfSF72uhbyu0ILw6OgMsLbRkAPAB1N2lzfJESBdrHxN3Ju_hi9UHx6jeQ2TnmoPGHfO-kFYebEJwsMYJ4k3-LU1KgIMsNuTbhFpMDZJ_XsJAVtK14-3dnrQWqlTQcy3oKLtQuj_MsNK_YYnvIOt6kk-ZgQo=](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGJsEjV6EgkuWJJq002Fo56MyrhSkDR1hfPaqBc9JnxqojIt3fKtTrPOXFZMfSF72uhbyu0ILw6OgMsLbRkAPAB1N2lzfJESBdrHxN3Ju_hi9UHx6jeQ2TnmoPGHfO-kFYebEJwsMYJ4k3-LU1KgIMsNuTbhFpMDZJ_XsJAVtK14-3dnrQWqlTQcy3oKLtQuj_MsNK_YYnvIOt6kk-ZgQo=)
> 
> *   **Dicas Gerais:**
> 
>     *   **Técnicas de Relaxamento para Estudantes (Artigo):** O Blog Grupo Integrado apresenta técnicas de relaxamento para estudantes, incluindo respiração profunda e alongamento. [Blog Grupo Integrado] → [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFjxrP9k4HDB8ntvu_38JS1gfNP-yF7PJvWAdFw9o9r_t3qT-oRTZjZyCvLKdGruirouDtDQA4zKbqphl7ei5m2-hO4OIx5ICyqmQEMB6l6FQc4Wdjt2pzsCmKOtCXsw7mYy1mUWiQwRH4wkKN_oQ2-v52oeLphxw0IzjqGV2owrnw=](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFjxrP9k4HDB8ntvu_38JS1gfNP-yF7PJvWAdFw9o9r_t3qT-oRTZjZyCvLKdGruirouDtDQA4zKbqphl7ei5m2-hO4OIx5ICyqmQEMB6l6FQc4Wdjt2pzsCmKOtCXsw7mYy1mUWiQwRH4wkKN_oQ2-v52oeLphxw0IzjqGV2owrnw=)
>     *   **Como Lidar com a Ansiedade Pré-Prova (Artigo):** A UNE explora técnicas de relaxamento e estratégias para controlar a ansiedade pré-prova, como exercícios físicos e respiração profunda. [UNE] → [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGOu-b0jhKGp60fFsqDNEZHF3S7yEq3UkQpNjMvAGTQ-YGgT5m83JC544N8CpGcIk-iumsctsa3PA5s1_JBTQbWqbilla9vie36JLG57L6jog9kp_72pAK3C9XICf1dAPj3ir6fGbDihuzUt9OaIms-XCucxqU2_AU675kctUW_wWx4x8tRXEX2Cme7g8iiFqrenqiiMN4=](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGOu-b0jhKGp60fFsqDNEZHF3S7yEq3UkQpNjMvAGTQ-YGgT5m83JC544N8CpGcIk-iumsctsa3PA5s1_JBTQbWqbilla9vie36JLG57L6jog9kp_72pAK3C9XICf1dAPj3ir6fGbDihuzUt9OaIms-XCucxqU2_AU675kctUW_wWx4x8tRXEX2Cme7g8iiFqrenqiiMN4=)
> 
> Lembre-se que a preparação adequada e o autocuidado são essenciais para reduzir a ansiedade e melhorar o desempenho nos estudos e nas provas.
> 
